In [1]:
import sys
import os

# add parent directory to path
library_path = os.path.abspath('..')
if library_path not in sys.path:
    sys.path.append(library_path)

from luxgiant_dstream.plots import draw_trumpet_plot

In [2]:
import pandas as pd

path_to_project = '/mnt/0A2AAC152AABFBB7/data/gwasResult/'
file = 'annotated_normalized_combined_R2_0.3.dose_step2_sex_pheno-glm.PHENO1.glm.logistic.hybrid'
freq = 'ds_results.frq'

df = pd.read_csv(
    os.path.join(path_to_project, file),
    sep='\t'
)

In [3]:
df['#CHROM'] = df['#CHROM'].astype(int)

In [4]:
df_freq = pd.read_csv(
    os.path.join(path_to_project, freq),
    sep='\s+',
    usecols=['SNP', 'MAF']
)

In [5]:
df_freq.columns

Index(['SNP', 'MAF'], dtype='object')

In [6]:
df = pd.merge(df, df_freq, left_on='ID', right_on='SNP')
n_cases = 4806
n_controls = 6364
df_annot = pd.read_csv(os.path.join(path_to_project, 'snps_annotated.csv'))

In [7]:
# rename columns to match GWASlab format
rename_cols = {
            'SNP': 'rsID',
            '#CHROM': 'CHR',
            'POS': 'POS',
            'A1' : 'EA',
            'ALT' : 'NEA',
            'A1_FREQ': 'EAF',
            'b'  : 'BETA',
            'se' : 'SE',
            'Z_STAT': 'Z',
            'p'  : 'P',
            'MAF': 'MAF'
        }

df = df.rename(columns=rename_cols)

In [8]:
df.columns

Index(['CHR', 'POS', 'ID', 'REF', 'NEA', 'PROVISIONAL_REF?', 'EA', 'OMITTED',
       'EAF', 'FIRTH?', 'TEST', 'N', 'BETA', 'SE', 'L95', 'U95', 'Z', 'P',
       'ERRCODE', 'rsID', 'MAF'],
      dtype='object')

In [9]:
df['SNPID'] = df['CHR'].astype(str) +':'+ df['POS'].astype(str) +':'+ df['EA'].astype(str) +':'+ df['NEA'].astype(str)
        
# find SNP IDs to highlight
if df_annot is not None:
    df_annot = df_annot.merge(
        df[['rsID', 'SNPID', 'CHR', 'POS']], 
        left_on='SNP', 
        right_on='rsID', 
        how='inner'
    )
    df_annot = df_annot.drop(columns=['SNP'])

# power curves thresholds
ts=[0.2,0.4,0.6,0.8, 0.9]

trumpet = draw_trumpet_plot(
    df_gwas    =df,
    plot_dir   =path_to_project,
    mode       ="b",
    xscale     ='nonlog',
    n_matrix   =3000,
    ts         =ts,
    ncase      =n_cases, 
    ncontrol   =n_controls, 
    prevalence =0.05,
    figargs    ={"figsize": (20, 16), "dpi": 600},
    font_family='DejaVu Sans',
    highlight  =df_annot['SNPID'].tolist(),
    highlight_windowkb = 0.01,
    anno       ="GENENAME",
    build      ="38",
    anno_set   =df_annot['SNPID'].tolist(),
    anno_style ="expand",
    ylim       =(-df['BETA'].abs().max()*1.5, df['BETA'].abs().max()*1.5),
    xlim       =(df['MAF'].min()*0.5,0.52),
    anno_args  ={'bbox':dict(boxstyle='round,pad=0.3', edgecolor='black', facecolor='#f0f0f0', alpha=0.8)}
)

2024/10/26 22:50:18 Start to create trumpet plot...
2024/10/26 22:50:18  -Settings:
2024/10/26 22:50:18   -Mode: b
2024/10/26 22:50:18   -N_CASE: 4806
2024/10/26 22:50:18   -N_CONTROL: 6364
2024/10/26 22:50:18   -PREVALENCE: 0.05
2024/10/26 22:50:18   -BETA: BETA
2024/10/26 22:50:18   -Significance level: 5e-08
2024/10/26 22:50:18   -Power thresholds: [0.2, 0.4, 0.6, 0.8, 0.9]
2024/10/26 22:50:18   -Power line smoothness: 3000
2024/10/26 22:50:18  -Excluding variants with P values > 5e-08
2024/10/26 22:50:18  -Plotting 1057 variants...
2024/10/26 22:50:18  -Flipping 157 variants...
2024/10/26 22:50:18  -Updating eaf-beta matrix...
2024/10/26 22:50:18  -GRR is approximated using OR. For prevalence < 10%, GRR is very similar to OR....
2024/10/26 22:50:20  -Extracting eaf-beta combinations with power = 0.2...
2024/10/26 22:50:20  -Updating eaf-beta matrix...
2024/10/26 22:50:20  -GRR is approximated using OR. For prevalence < 10%, GRR is very similar to OR....
2024/10/26 22:50:21  -Extrac

2024/10/26 22:50:32 Finished creating trumpet plot!
